# Network for the 17 MT-related genes DE including MTUS1

This notebook provides a reproducible pipeline to build, filter, and annotate a network
around ATIP3/MTUS1 from sources such as OmniPath/SIGNOR, combined with gene lists
(411 MT, 280 MT-related, 17 genes of interest, etc.).

1. Configuration & dependencies
2. Big Network loading
3. Extract 17-gene network

## 1) Configuration & dependencies

In [ ]:
import sys,  random
from pathlib import Path
import numpy as np
import pandas as pd

from pypath.core import network  #pypath.core: contains the database classes e.g. network, complex, annotations, etc
from pypath.resources import network as netres
from pypath.utils import mapping

import pypath
from pypath import omnipath #pypath.omnipath: higher level applications, e.g. a database manager, a web server

print(pypath.__version__)

SEED = 42
random.seed(SEED)

PROJ = Path.cwd().resolve().parent

DATA = PROJ / 'data'      # put your inputs here (CSV, lists, mappings, etc.)
OUT  = PROJ / 'out'       # outputs
SRC  = PROJ / 'src'       # optional local code (wrappers, helpers)


OUT.mkdir(exist_ok=True, parents=True)

if SRC.exists():
    sys.path.insert(0, str(SRC))
else:
    print("[WARN] SRC not found:", SRC)

try:
    from pypath_wrapper import Wrap_net  # provide in src/ if needed
except ImportError as e:
    print("No file pypath_wrapper.py:", e)


[2025-08-29 20:17:49] [curl] Module `pysftp` not available. Only downloading of a small number of resources relies on this module. Please install by PIP if it is necessary for you.
0.14.48


## 2) Big Network loading

In [2]:
w = Wrap_net() #here we create the wrap_net object, which is the same as Network() but with a couple of functions more

In [3]:
#here we load the database to populate the wrap_net object using the "netres" object
netres.omnipath
#in the output of the cells you can visualize the database that will populate your wrapnet object

{'trip': <NetworkResource: TRIP (post_translational, activity_flow)>,
 'spike': <NetworkResource: SPIKE (post_translational, activity_flow)>,
 'signalink3': <NetworkResource: SignaLink3 (post_translational, activity_flow)>,
 'guide2pharma': <NetworkResource: Guide2Pharma (post_translational, activity_flow)>,
 'ca1': <NetworkResource: CA1 (post_translational, activity_flow)>,
 'arn': <NetworkResource: ARN (post_translational, activity_flow)>,
 'nrf2': <NetworkResource: NRF2ome (post_translational, activity_flow)>,
 'macrophage': <NetworkResource: Macrophage (post_translational, activity_flow)>,
 'death': <NetworkResource: DeathDomain (post_translational, activity_flow)>,
 'pdz': <NetworkResource: PDZBase (post_translational, activity_flow)>,
 'signor': <NetworkResource: SIGNOR (post_translational, activity_flow)>,
 'adhesome': <NetworkResource: Adhesome (post_translational, activity_flow)>,
 'icellnet': <NetworkResource: ICELLNET (post_translational, activity_flow)>,
 'celltalkdb': <Net

In [ ]:
#to load the previous databses just do:
w.load(netres.pathway)
w.load(netres.pathway_all)
#the load methods, loads the databases contained in the netres object

In [ ]:
#now let's check its dimensions
w

<Network: 16137 nodes, 167967 interactions>

In [ ]:
#and let's have a look to the interactions ( or nodes)
w.interactions #w.nodes

{(<Entity: TRPC1>,
  <Entity: KCNMA1>): <Interaction: TRPC1 ============= KCNMA1 [Evidences: TRIP (2 references)]>,
 (<Entity: TRPC1>,
  <Entity: PPP3CA>): <Interaction: TRPC1 ============= PPP3CA [Evidences: TRIP (1 references)]>,
 (<Entity: CALM3>,
  <Entity: TRPC1>): <Interaction: CALM3 =======(-)==> TRPC1 [Evidences: TRIP (3 references)]>,
 (<Entity: CALM1>,
  <Entity: TRPC1>): <Interaction: CALM1 =======(-)==> TRPC1 [Evidences: TRIP (3 references)]>,
 (<Entity: CALM2>,
  <Entity: TRPC1>): <Interaction: CALM2 =======(-)==> TRPC1 [Evidences: TRIP (3 references)]>,
 (<Entity: CASP1>,
  <Entity: TRPC1>): <Interaction: CASP1 ============= TRPC1 [Evidences: TRIP (1 references)]>,
 (<Entity: TRPC1>,
  <Entity: CACNA1C>): <Interaction: TRPC1 ============= CACNA1C [Evidences: TRIP (1 references)]>,
 (<Entity: TRPC1>,
  <Entity: CAV1>): <Interaction: TRPC1 <=(+)======== CAV1 [Evidences: TRIP (7 references)]>,
 (<Entity: TRPC1>,
  <Entity: CAV3>): <Interaction: TRPC1 ============= CAV3 [Evid

In [ ]:
w.get_neighbours("MAPRE1")

['CEP57',
 'MCC',
 'TUBB4B',
 'CEP72',
 'TUBA4A',
 'AKAP9',
 'PRKAR2B',
 'SDCCAG8',
 'CNTRL',
 'SFI1',
 'HLA-B',
 'CLIP1',
 'CCP110',
 'PPP2R1A',
 'CEP63',
 'CLASP2',
 'YWHAE',
 'CEP135',
 'ALMS1',
 'TERF1',
 'CEP70',
 'CEP290',
 'NEDD1',
 'CLASP1',
 'CEP250',
 'CKAP5',
 'PLK1',
 'PCM1',
 'TRAF6',
 'CEP152',
 'CENPJ',
 'YWHAG',
 'EPB41',
 'CSNK1E',
 'CEP43',
 'PCNT',
 'CEP76',
 'TUBG1',
 'DYNC1H1',
 'NDE1',
 'NEK2',
 'CSH1',
 'DCTN2',
 'OFD1',
 'HSP90AA1',
 'DCTN3',
 'GH1',
 'DYNC1I2',
 'APC',
 'CEP131',
 'CEP164',
 'PAFAH1B1',
 'EIF1B',
 'IKBKE',
 'TUBA1A',
 'CSNK1D',
 'CDK5RAP2',
 'ACTR1A',
 'NINL',
 'HLA-C',
 'SSNA1',
 'TUBB4A',
 'PLK4',
 'DYNLL1',
 'ODF2',
 'TUBB',
 'CEP78',
 'CEP41',
 'DCTN1',
 'CDK1',
 'CETN2',
 'PRKACA',
 'HAUS2']

In [ ]:
w.get_neighbours("MTUS1")

['PARP1']

In [ ]:
ia = w.interaction('MAPK1', 'MAPT')
print(ia)

ia.get_references()


{"effect":"+","type":"Phosphorylation","mechanism":["phosphorylation"]}


{<Reference: 10737616>, <Reference: 7488031>, <Reference: 8166686>}

In [ ]:
# Make a dataframe
w.make_df()
df_w = w.df
df_w.shape

(179636, 10)

## 3) Extract 17 MT_Rel genes

### a) Define the gene list

In [ ]:
genes_17 = ["ASPM","AURKB","GTSE1","KIF11","KIF14","KIF15","KIF18B","KIF20A","KIF2C","KIF4A","KIFC1","MAPT","MAST4","MTUS1","RACGAP1","STMN1","TPX2"]

# we can have easily uniprot

uniprot_17 = []
for gene in genes_17:
    uniprot = list(mapping.map_name(gene, 'genesymbol', 'uniprot'))
    genesymbol = list(mapping.map_name(uniprot[0], 'uniprot', 'genesymbol'))
    print(uniprot, genesymbol)
    uniprot_17.append(uniprot[0])


['Q8IZT6'] ['ASPM']
['Q96GD4'] ['AURKB']
['Q9NYZ3'] ['GTSE1']
['P52732'] ['KIF11']
['Q15058'] ['KIF14']
['Q9NS87'] ['KIF15']
['Q86Y91'] ['KIF18B']
['O95235'] ['KIF20A']
['Q99661'] ['KIF2C']
['O95239'] ['KIF4A']
['Q9BW19'] ['KIFC1']
['P10636'] ['MAPT']
['O15021'] ['MAST4']
['Q9ULD2'] ['MTUS1']
['Q9H0H5'] ['RACGAP1']
['P16949'] ['STMN1']
['Q9ULW0'] ['TPX2']


### b) First Network with 17-gene neighbours

In [ ]:
for i in df_w.index:
    df_w.loc[i,'node_present'] = (df_w.loc[i,'id_a'] in uniprot_17) or  (df_w.loc[i,'id_b'] in uniprot_17)

df_17 = df_w[df_w['node_present'] == True]
df_17


,id_a,id_b,type_a,type_b,directed,effect,type,dmodel,sources,references,node_present
123,P16949,Q9UL62,protein,protein,False,0,post_translational,{activity_flow},{TRIP},NaN,True
3589,P09874,Q96GD4,protein,protein,True,-1,post_translational,{activity_flow},"{SPIKE_LC, SPIKE}",NaN,True
3665,O95239,O43663,protein,protein,True,1,post_translational,{activity_flow},"{SPIKE, SPIKE_LC, SIGNOR}",NaN,True
3771,P04637,Q9NYZ3,protein,protein,True,1,post_translational,{activity_flow},"{SPIKE_LC, Wang, SPIKE}",NaN,True
3772,Q9NYZ3,P04637,protein,protein,True,-1,post_translational,{activity_flow},"{SPIKE_LC, SPIKE}",NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
178501,P10636,P23443,protein,protein,False,0,post_translational,{activity_flow},{SPIKE_LC},NaN,True
178816,P62993,Q9ULW0,protein,protein,False,0,post_translational,{activity_flow},{SPIKE_LC},NaN,True
178873,Q16537,Q9H0H5,protein,protein,False,0,post_translational,{activity_flow},{SPIKE_LC},NaN,True
179246,P16949,Q99828,protein,protein,False,0,post_translational,{activity_flow},{SPIKE_LC},NaN,True


In [ ]:
# Add protein names

for i in df_17.index:
    if df_17.loc[i,"type_a"] == "protein":
        df_17.loc[i,"HGNC_a"] = list(mapping.map_name(df_17.loc[i,"id_a"], 'uniprot', 'genesymbol'))[0]
    else: #complex
        l = "COMPLEX:"
        for t in df_17.loc[i,"id_a"]:
            l+="_"+str(list(mapping.map_name(t, 'uniprot', 'genesymbol'))[0])
        df_17.loc[i,"HGNC_a"] = l

for i in df_17.index:
    if df_17.loc[i,"type_b"] == "protein":
        df_17.loc[i,"HGNC_b"] = list(mapping.map_name(df_17.loc[i,"id_b"], 'uniprot', 'genesymbol'))[0]
    else: #complex
        l = "COMPLEX:"
        for t in df_17.loc[i,"id_b"]:
            l+="_"+str(list(mapping.map_name(t, 'uniprot', 'genesymbol'))[0])
        df_17.loc[i,"HGNC_b"] = l

# remove when sources = {Wang}
df_17 = df_17[df_17['sources'] != {'Wang'}]

# to find our 17 genes
for i in df_17.index:
    df_17.loc[i,'node_source_present'] = (df_17.loc[i,'HGNC_a'] in genes_17)
    df_17.loc[i,'node_end_present'] =  (df_17.loc[i,'HGNC_b'] in genes_17)


# remove colums type	
df_17 = df_17.drop(columns=['type_a', 'type_b','directed','type','dmodel',  'references'])
df_17

/var/folders/zg/y03kdtc553xg5q1ct59rj44c0000gn/T/ipykernel_96270/2583043574.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_17.loc[i,"HGNC_a"] = list(mapping.map_name(df_17.loc[i,"id_a"], 'uniprot', 'genesymbol'))[0]
/var/folders/zg/y03kdtc553xg5q1ct59rj44c0000gn/T/ipykernel_96270/2583043574.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_17.loc[i,"HGNC_b"] = list(mapping.map_name(df_17.loc[i,"id_b"], 'uniprot', 'genesymbol'))[0]
/var/folders/zg/y03kdtc553xg5q1ct59rj44c0000gn/T/ipykernel_9627

,id_a,id_b,effect,sources,node_present,HGNC_a,HGNC_b,node_source_present,node_end_present
123,P16949,Q9UL62,0,{TRIP},True,STMN1,TRPC5,True,False
3589,P09874,Q96GD4,-1,"{SPIKE_LC, SPIKE}",True,PARP1,AURKB,False,True
3665,O95239,O43663,1,"{SPIKE, SPIKE_LC, SIGNOR}",True,KIF4A,PRC1,True,False
3771,P04637,Q9NYZ3,1,"{SPIKE_LC, Wang, SPIKE}",True,TP53,GTSE1,False,True
3772,Q9NYZ3,P04637,-1,"{SPIKE_LC, SPIKE}",True,GTSE1,TP53,True,False
...,...,...,...,...,...,...,...,...,...
178501,P10636,P23443,0,{SPIKE_LC},True,MAPT,RPS6KB1,True,False
178816,P62993,Q9ULW0,0,{SPIKE_LC},True,GRB2,TPX2,False,True
178873,Q16537,Q9H0H5,0,{SPIKE_LC},True,PPP2R5E,RACGAP1,False,True
179246,P16949,Q99828,0,{SPIKE_LC},True,STMN1,CIB1,True,False


In [ ]:
# save to csv (use Cytoscape to visualize the graph and its properties)
#df_17.to_csv("df_graph_17.csv")

#### there are too many not connected neighbours

In [ ]:
# node degree

import numpy as np
# we want the nodes that appear in only one line
df_count_a = pd.DataFrame(df_17["HGNC_a"].value_counts())
df_count_b = pd.DataFrame(df_17["HGNC_b"].value_counts())

df_count = pd.merge(df_count_a, df_count_b, how='outer', left_index=True, right_index=True)
df_count["count"] = np.nansum(df_count, axis=1)

df_count = df_count.sort_values(by=['count'], ascending=False)

# table of nodes with degree 1
df1 = df_count[df_count["count"]==1].sort_index()

# we want the nodes that appear in only one line
df1 = df1.reset_index()
df1 = df1.rename(columns={"index": "HGNC_a"})



In [ ]:
# we look if these nodes are in our list of 17 genes
df1[df1["HGNC_a"]=="MAPRE1"] # not present
df1[df1["HGNC_a"]=="PSRC1"] # not present
df1[df1["HGNC_a"]=="KIF23"]
df1[df1["HGNC_a"]=="KIF2A"]

,HGNC_a,count_x,count_y,count
67,KIF2A,NaN,1.0,1.0


#### Keep genes in 280 MT-Rel genes

Before removing the nodes, I will look at their normal vs. breast cancer (BC) expression and whether they are in the 280 MT-related genes.
I only keep those that are in the 280 differentially expressed MT-related genes + CENPE that is not in the 411 MT

In [ ]:
# we look if the nodes in df_nodes are in df_280
df_280 = pd.read_csv('../data/280-MT-related-genes.csv',sep=';')
df_280.head()

,Symbol,Entrez Gene Name,Entrez Gene ID for Human,Unnamed: 3,Unnamed: 4
0,AGTPBP1,ATP/GTP binding protein 1,23287,NaN,NaN
1,AMER2,APC membrane recruitment protein 2,219287,NaN,NaN
2,ANXA11,annexin A11,311,NaN,NaN
3,ANXA5,annexin A5,308,NaN,NaN
4,APC,adenomatous polyposis coli,324,NaN,NaN


In [ ]:
df1["dans_280"] = df1["HGNC_a"].apply(lambda x: x in df_280["Symbol"].values)

df2 = pd.concat([df1[df1["dans_280"]], df1[df1["HGNC_a"]=="CENPE"]])
df2.head()

,HGNC_a,count_x,count_y,count,dans_280
16,CDK2,1.0,NaN,1.0,True
17,CDK4,1.0,NaN,1.0,True
29,CLASP1,NaN,1.0,1.0,True
39,CSNK1D,1.0,NaN,1.0,True
41,DCTN1,NaN,1.0,1.0,True


#### Add normal vs. breast cancer (BC) log2FC 

Up or down regulated in differential gene expression analysis in Tumor vs Normal tissues. https://tnmplot.com/analysis/

In [ ]:
# load the dataframe
df_results =   pd.read_csv("../data/df_DEG_normal_vs_tumor.csv", index_col=0)
df_results.head()

,HGNC_a,p-value,log2FC,log2FC2
0,ASPM,9.670000e-20,12.06,14.66
1,ATM,2.680000e-07,0.72,0.77
2,AURKA,5.290000e-20,8.21,7.01
3,AURKB,5.980000e-19,8.05,8.05
4,CAMK2A,4.430000e-08,0.24,2.25


In [ ]:
# we merge df_results and df2 on the HGNC_a column
df3 = pd.merge(df_results, df2, how='inner', left_on='HGNC_a', right_on='HGNC_a')
df3["up_down"] = df3.apply(lambda x: 3 if (x[1]<0.05 and x[3] >=5) else 2 if (x[1]<0.05 and x[3]>=2) else -2 if (x[1]<0.05 and x[3]<=0.5)  else -1 if (x[1]<0.05 and x[3]<=2/3)else 0, axis=1)
df3 = df3[df3["up_down"] != 0]
df3

,HGNC_a,p-value,log2FC,log2FC2,count_x,count_y,count,dans_280,up_down
1,CENPE,5.320000e-19,6.28,5.00,NaN,1.0,1.0,False,3
5,DIAPH2,2.090000e-08,0.58,0.55,NaN,1.0,1.0,True,-1
6,FYN,9.420000e-09,0.58,0.56,1.0,NaN,1.0,True,-1
8,KIF23,1.150000e-19,5.92,5.42,NaN,1.0,1.0,True,3
14,TUBB3,6.750000e-19,11.03,7.75,NaN,1.0,1.0,True,3
21,LRRK2,3.770000e-08,0.72,0.54,1.0,NaN,1.0,True,-1
23,MARK1,4.790000e-10,0.48,0.39,1.0,NaN,1.0,True,-2
25,PRKAA1,8.240000e-07,0.72,0.66,1.0,NaN,1.0,True,-1
31,TTBK1,4.450000e-08,3.05,2.00,1.0,NaN,1.0,True,2


In [ ]:
# we merge df_results and df1 to find the nodes in df1 that are not in df_results
df4 = pd.merge(df_results, df2, how='outer', left_on='HGNC_a', right_on='HGNC_a')
# not in df1
# print the genes not in df1


df4 = df4[df4["p-value"].isna()]
print(df4.shape)
for g in df4["HGNC_a"].to_list():
    print(g)


(0, 8)


#### I clean df1 of the nodes I don't want to remove

1. I remove the degree 1 nodes, which are in 280 mT-related genes and differentially expressed (to keep them in the final network)

2. I remove CDK4 (to connect with ASPM), the KIF2C_KIF18B complex (2 DEGs), and KIF2A (to connect MTUS1)


In [ ]:
print(df1.shape)
for g in df3["HGNC_a"].to_list():
    df1 = df1[df1["HGNC_a"]!=g]
print(df1.shape)

(166, 5)
(157, 5)


In [ ]:
df1 = df1[df1["HGNC_a"]!="CDK4"]

df1 = df1[df1["HGNC_a"]!="COMPLEX:_KIF2C_KIF18B"]

df1 = df1[df1["HGNC_a"]!="KIF2A"]
print(df1.shape)


(154, 5)


#### I remove all nodes with only 1 neighbor from df_17 that are in df1

In [ ]:
print(df_17.shape)
df_17_2 = df_17[~df_17["HGNC_a"].isin(df1["HGNC_a"]) & ~df_17["HGNC_b"].isin(df1["HGNC_a"])]
print(df_17_2.shape)
df_17_2[df_17_2["HGNC_b"]=="DIAPH2"]


(253, 9)
(99, 9)


,id_a,id_b,effect,sources,node_present,HGNC_a,HGNC_b,node_source_present,node_end_present
81720,Q96GD4,O60879,1,{SIGNOR},True,AURKB,DIAPH2,True,False


In [ ]:
#df_17_2.to_csv("df_graph_17_qq_1voisin_2.csv")

#### there are still nodes with double arrows, I remove them:

In [ ]:
A_enlever = ["H3C12","TUBA1B","GSK3B","PPP2CA","CENPA","H3C4","INCENP","BIRC5","DES","GFAP","NDC80","COMPLEX:_PPP1CA_PPP1CB_PPP1CC"]
df_17_3 = df_17_2[~df_17_2["HGNC_a"].isin(A_enlever) & ~df_17_2["HGNC_b"].isin(A_enlever)]
#df_17_3.to_csv("df_graph_17_qq_1voisin_3.csv")


#### "manual" addition (MTUS1 interacts with "KIF2A") 

MTUS1 interacts with KIF2A https://pubmed.ncbi.nlm.nih.gov/32789689/


#### "manual" addition of KIF18B (interacts with KIF2C) (and MAPRE1, no because Mapre1 is not in the network)

**KIF18B**    
* Interacts with MAPRE1; this interaction is required for efficient accumulation at microtubule plus ends. https://pubmed.ncbi.nlm.nih.gov/21737685/ <br>
* Interacts with KIF2C at microtubule tips; this interaction increases the affinity of both partners for microtubule plus ends and is required for robust microtubule depolymerization. https://www.uniprot.org/uniprotkb/Q86Y91/entry  <br>
            (KIF2C phosphorylation by AURKA or AURKB strongly reduces KIF18B-binding)  <br>

In [ ]:
#add MTUS1
df5 = pd.DataFrame({'id_a':["O00139"] , 'id_b': ["Q9ULD2"], 'effect': [-2],'sources': 'main', 'HGNC_a': ["KIF2A"], 'HGNC_b': ["MTUS1"]})
df_17_3 = pd.concat([df_17_3, df5], ignore_index=True)
#add KIF18B
df5 = pd.DataFrame({'id_a':["Q99661"] , 'id_b': ["Q86Y91"], 'effect': [-2],'sources': 'main',  'HGNC_a': ["KIF2C"], 'HGNC_b':  ["KIF18B"]})
df_17_3 = pd.concat([df_17_3, df5], ignore_index=True)
df_17_3[df_17_3["HGNC_b"]=="DIAPH2"]


,id_a,id_b,effect,sources,node_present,HGNC_a,HGNC_b,node_source_present,node_end_present
53,Q96GD4,O60879,1,{SIGNOR},True,AURKB,DIAPH2,True,False


In [ ]:
df_17_3.shape

(80, 9)

#### we take all the nodes of the previous network obtained and we connect the nodes which are not in the initial list of 20 genes between them by searching in the database all the nodes at distance 1

In [ ]:
# all genes in the network except COMPLEX
all_nodes  = list(set(df_17_3["HGNC_a"].values) | set(df_17_3["HGNC_b"].values))
print("nodes number (with complex)",len(all_nodes))
for elt in all_nodes:
    if 'COMPLEX'  in elt:
        all_nodes.remove(elt)
print("nodes number (without complex)",len(all_nodes))
# we can have easily uniprot
uniprot_all_nodes = []
for gene in all_nodes:
    uniprot = list(mapping.map_name(gene, 'genesymbol', 'uniprot'))
    try : 
        genesymbol = list(mapping.map_name(uniprot[0], 'uniprot', 'genesymbol'))
        uniprot_all_nodes.append(uniprot[0])
    except:
        print(uniprot, genesymbol)
        uniprot_all_nodes.append("P29323")
    
print(uniprot_all_nodes)

nodes number (with complex) 50
nodes number (without complex) 47
['O14965', 'Q5TCY1', 'O95239', 'Q13315', 'Q9NYZ3', 'O14578', 'Q13554', 'P06241', 'Q9NS87', 'Q8IZT6', 'O60879', 'Q86Y91', 'P62993', 'P16949', 'P31946', 'Q9ULW0', 'P68431', 'P45983', 'P11802', 'P53350', 'Q9H0H5', 'P08670', 'P29323', 'Q13509', 'Q00535', 'Q96GD4', 'P52732', 'O95235', 'Q9BW19', 'O00139', 'Q9ULD2', 'P10636', 'Q02241', 'Q9UQM7', 'P04637', 'O15021', 'Q02224', 'Q9P0L2', 'Q5S007', 'Q15058', 'P28482', 'P06493', 'O43663', 'Q13131', 'Q99661', 'Q13153', 'P17612']


In [ ]:
for i in df_w.index:
    df_w.loc[i,'node_present_all'] = (df_w.loc[i,'id_a'] in uniprot_all_nodes) or  (df_w.loc[i,'id_b'] in uniprot_all_nodes)

In [ ]:
# new net redo with distance = 1 and "all nodes"
df_graph_all_nodes = df_w[df_w['node_present_all'] == True]
df_graph_all_nodes

,id_a,id_b,type_a,type_b,directed,effect,type,dmodel,sources,references,node_present,node_present_all
101,P06241,Q9UBN4,protein,protein,False,0,post_translational,{activity_flow},{TRIP},NaN,False,True
123,P16949,Q9UL62,protein,protein,False,0,post_translational,{activity_flow},{TRIP},NaN,True,True
144,P06241,Q9Y210,protein,protein,True,1,post_translational,{activity_flow},{TRIP},NaN,False,True
145,P06241,Q9Y210,protein,protein,False,0,post_translational,{activity_flow},{SPIKE_LC},NaN,False,True
155,P17612,Q9Y210,protein,protein,False,0,post_translational,{activity_flow},{TRIP},NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
179536,P08670,Q96C92,protein,protein,False,0,post_translational,{activity_flow},{SPIKE_LC},NaN,False,True
179551,P26651,P31946,protein,protein,False,0,post_translational,{activity_flow},{SPIKE_LC},NaN,False,True
179564,P31946,Q92538,protein,protein,False,0,post_translational,{activity_flow},{SPIKE_LC},NaN,False,True
179569,Q02241,Q9NR09,protein,protein,False,0,post_translational,{activity_flow},{SPIKE_LC},NaN,False,True


In [ ]:
# add name of the proteins
for i in df_graph_all_nodes.index:
    if df_graph_all_nodes.loc[i,"type_a"] == "protein":
        df_graph_all_nodes.loc[i,"HGNC_a"] = list(mapping.map_name(df_graph_all_nodes.loc[i,"id_a"], 'uniprot', 'genesymbol'))[0]
    else: #complex
        l = "COMPLEX:"
        for t in df_graph_all_nodes.loc[i,"id_a"]:
            l+="_"+str(list(mapping.map_name(t, 'uniprot', 'genesymbol'))[0])
        df_graph_all_nodes.loc[i,"HGNC_a"] = l

for i in df_graph_all_nodes.index:
    if df_graph_all_nodes.loc[i,"type_b"] == "protein":
        df_graph_all_nodes.loc[i,"HGNC_b"] = list(mapping.map_name(df_graph_all_nodes.loc[i,"id_b"], 'uniprot', 'genesymbol'))[0]
    else: #complex
        l = "COMPLEX:"
        for t in df_graph_all_nodes.loc[i,"id_b"]:
            try:
                l+="_"+str(list(mapping.map_name(t, 'uniprot', 'genesymbol'))[0])
            except:
                l = "pb"
        df_graph_all_nodes.loc[i,"HGNC_b"] = l

# remove when sources = {Wang}
df_graph_all_nodes = df_graph_all_nodes[df_graph_all_nodes['sources'] != {'Wang'}]

# to find our 17 genes DE
#for i in df_graph_all_nodes.index:
#    df_graph_all_nodes.loc[i,'node_source_present'] = (df_graph_all_nodes.loc[i,'HGNC_a'] in genes_17)
#    df_graph_all_nodes.loc[i,'node_end_present'] =  (df_graph_all_nodes.loc[i,'HGNC_b'] in genes_17)

# remove colums type	
df_graph_all_nodes = df_graph_all_nodes.drop(columns=['type_a', 'type_b','directed','type','dmodel',  'references'])
df_graph_all_nodes

/var/folders/zg/y03kdtc553xg5q1ct59rj44c0000gn/T/ipykernel_96270/1186719133.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graph_all_nodes.loc[i,"HGNC_a"] = list(mapping.map_name(df_graph_all_nodes.loc[i,"id_a"], 'uniprot', 'genesymbol'))[0]
/var/folders/zg/y03kdtc553xg5q1ct59rj44c0000gn/T/ipykernel_96270/1186719133.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graph_all_nodes.loc[i,"HGNC_b"] = list(mapping.map_name(df_graph_all_nodes.loc[i,"id_b"], 'uniprot', 'genesymbol'))[0]


,id_a,id_b,effect,sources,node_present,node_present_all,HGNC_a,HGNC_b
101,P06241,Q9UBN4,0,{TRIP},False,True,FYN,TRPC4
123,P16949,Q9UL62,0,{TRIP},True,True,STMN1,TRPC5
144,P06241,Q9Y210,1,{TRIP},False,True,FYN,TRPC6
145,P06241,Q9Y210,0,{SPIKE_LC},False,True,FYN,TRPC6
155,P17612,Q9Y210,0,{TRIP},False,True,PRKACA,TRPC6
...,...,...,...,...,...,...,...,...
179536,P08670,Q96C92,0,{SPIKE_LC},False,True,VIM,ENTR1
179551,P26651,P31946,0,{SPIKE_LC},False,True,ZFP36,YWHAB
179564,P31946,Q92538,0,{SPIKE_LC},False,True,YWHAB,GBF1
179569,Q02241,Q9NR09,0,{SPIKE_LC},False,True,KIF23,BIRC6


In [ ]:
df_graph_all_nodes[df_graph_all_nodes["HGNC_b"]== "pb"]

,id_a,id_b,effect,sources,node_present,node_present_all,HGNC_a,HGNC_b


In [ ]:
df_graph_all_nodes[(df_graph_all_nodes["HGNC_a"]== "TP53") &(df_graph_all_nodes["HGNC_b"]== "ATM")]

,id_a,id_b,effect,sources,node_present,node_present_all,HGNC_a,HGNC_b
4434,P04637,Q13315,1,"{Cui2007, Wang}",False,True,TP53,ATM


In [ ]:
### we extract from this new df, the interactions whose nodes are in all_nodes (nodes of the old network)
df_graph_all_nodes_only = df_graph_all_nodes[df_graph_all_nodes['HGNC_a'].isin(all_nodes) & df_graph_all_nodes['HGNC_b'].isin(all_nodes)]
# we remove node_present	node_present_all
df_graph_all_nodes_only = df_graph_all_nodes_only.drop(columns=['node_present', 'node_present_all'])
df_graph_all_nodes_only

,id_a,id_b,effect,sources,HGNC_a,HGNC_b
453,P04637,O43663,-1,"{SPIKE_LC, Wang, SPIKE}",TP53,PRC1
1714,Q13315,P53350,-1,"{SPIKE_LC, SPIKE}",ATM,PLK1
2217,P17612,P68431,1,"{SPIKE_LC, SPIKE}",PRKACA,H3C7
3665,O95239,O43663,1,"{SPIKE, SPIKE_LC, SIGNOR}",KIF4A,PRC1
3771,P04637,Q9NYZ3,1,"{SPIKE_LC, Wang, SPIKE}",TP53,GTSE1
...,...,...,...,...,...,...
172718,Q13554,Q9UQM7,0,{SPIKE_LC},CAMK2B,CAMK2A
172831,P06241,P06493,0,{SPIKE_LC},FYN,CDK1
175266,P11802,Q8IZT6,0,{SPIKE_LC},CDK4,ASPM
176078,P31946,Q02241,0,{SPIKE_LC},YWHAB,KIF23


In [ ]:
# we merge df_graph_all_nodes_only and df_17_3
df_graph_all_nodes_only_merge_17genes_3 = pd.concat([df_graph_all_nodes_only, df_17_3], ignore_index=True)
df_graph_all_nodes_only_merge_17genes_3.drop(columns=['node_present','node_source_present', 'node_end_present'], inplace=True)
df_graph_all_nodes_only_merge_17genes_3 

,id_a,id_b,effect,sources,HGNC_a,HGNC_b
0,P04637,O43663,-1,"{SPIKE_LC, Wang, SPIKE}",TP53,PRC1
1,Q13315,P53350,-1,"{SPIKE_LC, SPIKE}",ATM,PLK1
2,P17612,P68431,1,"{SPIKE_LC, SPIKE}",PRKACA,H3C7
3,O95239,O43663,1,"{SPIKE, SPIKE_LC, SIGNOR}",KIF4A,PRC1
4,P04637,Q9NYZ3,1,"{SPIKE_LC, Wang, SPIKE}",TP53,GTSE1
...,...,...,...,...,...,...
214,P06493,Q9H0H5,0,{SPIKE_LC},CDK1,RACGAP1
215,P11802,Q8IZT6,0,{SPIKE_LC},CDK4,ASPM
216,P62993,Q9ULW0,0,{SPIKE_LC},GRB2,TPX2
217,O00139,Q9ULD2,-2,main,KIF2A,MTUS1


In [ ]:
# we remove all effects = 0
#df_graph_all_nodes_only_merge_17genes_3 = df_graph_all_nodes_only_merge_17genes_3[df_graph_all_nodes_only_merge_17genes_3['effect'] != 0]
#df_graph_all_nodes_only_merge_17genes_3

In [ ]:
# find all sources
list_sources = df_graph_all_nodes_only_merge_17genes_3['sources'].to_list()
set_sources = []
for elt in list_sources:
    for name in elt:
        if name not in set_sources :
            set_sources.append(name)
set_sources[:-4]

['SPIKE_LC',
 'Wang',
 'SPIKE',
 'SIGNOR',
 'Cui2007',
 'KEGG',
 'Macrophage',
 'CA1',
 'KEGG-MEDICUS',
 'Adhesome']

#### some interactions are in both directions (-1 and 1), we only keep those which are in SIGNOR

In [ ]:
# we make a new dataframe with unique pairs of hgnc_a and hgnc_b while keeping the different effects
df_graph_all_nodes_only_merge_17genes_3_unique = df_graph_all_nodes_only_merge_17genes_3.groupby(['HGNC_a','HGNC_b']).agg({'effect': lambda x: list(x), 'sources': lambda x: list(x)}).reset_index()
df_graph_all_nodes_only_merge_17genes_3_unique

,HGNC_a,HGNC_b,effect,sources
0,ATM,AURKA,[0],[{SPIKE_LC}]
1,ATM,KIF11,"[0, 0]","[{SPIKE_LC}, {SPIKE_LC}]"
2,ATM,KIF2C,"[0, 0]","[{SPIKE_LC}, {SPIKE_LC}]"
3,ATM,MAST4,"[0, 0]","[{SPIKE_LC}, {SPIKE_LC}]"
4,ATM,PLK1,[-1],"[{SPIKE_LC, SPIKE}]"
...,...,...,...,...
128,VIM,KIF15,"[0, 0]","[{SPIKE_LC}, {SPIKE_LC}]"
129,VIM,PAK1,"[1, 0]","[{Adhesome}, {SIGNOR}]"
130,VIM,PRKACA,[0],[{Adhesome}]
131,YWHAB,KIF23,[0],[{SPIKE_LC}]


In [ ]:
# we search in df_graph_all_nodes_only_merge_17genes_3_unique when effect is of size >= 1
df_graph_all_nodes_only_merge_17genes_3_unique["effect_size"] = df_graph_all_nodes_only_merge_17genes_3_unique["effect"].apply(lambda x: len(x))
df_graph_all_nodes_only_merge_17genes_3_unique[df_graph_all_nodes_only_merge_17genes_3_unique["effect_size"]>=2]

,HGNC_a,HGNC_b,effect,sources,effect_size
1,ATM,KIF11,"[0, 0]","[{SPIKE_LC}, {SPIKE_LC}]",2
2,ATM,KIF2C,"[0, 0]","[{SPIKE_LC}, {SPIKE_LC}]",2
3,ATM,MAST4,"[0, 0]","[{SPIKE_LC}, {SPIKE_LC}]",2
5,ATM,TP53,"[1, -1, 0]","[{SPIKE, SIGNOR, SPIKE_LC, Cui2007, KEGG}, {Cu...",3
6,ATM,TPX2,"[0, 0]","[{SPIKE_LC}, {SPIKE_LC}]",2
...,...,...,...,...,...
126,TPX2,KIF11,"[-1, -1]","[{SIGNOR}, {SIGNOR}]",2
127,TTBK1,MAPT,"[-1, -1]","[{SIGNOR}, {SIGNOR}]",2
128,VIM,KIF15,"[0, 0]","[{SPIKE_LC}, {SPIKE_LC}]",2
129,VIM,PAK1,"[1, 0]","[{Adhesome}, {SIGNOR}]",2


In [ ]:
# if all values in effect are the same, keep only one value
df_graph_all_nodes_only_merge_17genes_3_unique["effect_unique"] = df_graph_all_nodes_only_merge_17genes_3_unique["effect"].apply(lambda x: list(set(x)))
df_graph_all_nodes_only_merge_17genes_3_unique["effect_size_unique"] = df_graph_all_nodes_only_merge_17genes_3_unique["effect_unique"].apply(lambda x: len(x))
df_graph_all_nodes_only_merge_17genes_3_unique_2 = df_graph_all_nodes_only_merge_17genes_3_unique[df_graph_all_nodes_only_merge_17genes_3_unique["effect_size_unique"]>=2]
df_graph_all_nodes_only_merge_17genes_3_unique_2

,HGNC_a,HGNC_b,effect,sources,effect_size,effect_unique,effect_size_unique
5,ATM,TP53,"[1, -1, 0]","[{SPIKE, SIGNOR, SPIKE_LC, Cui2007, KEGG}, {Cu...",3,"[0, 1, -1]",3
12,AURKA,TP53,"[1, -1]","[{Wang, SIGNOR}, {SPIKE, SPIKE_LC, SIGNOR}]",2,"[1, -1]",2
18,AURKB,H3C7,"[1, -1, 1, -1]","[{SIGNOR}, {SPIKE_LC, SPIKE}, {SIGNOR}, {SPIKE...",4,"[1, -1]",2
22,AURKB,KIF2C,"[1, -1, 1, -1]","[{SPIKE, SPIKE_LC, SIGNOR}, {Wang, SIGNOR}, {S...",4,"[1, -1]",2
27,CAMK2A,MAPT,"[1, -1, 1, -1]","[{Wang, KEGG}, {SIGNOR}, {Wang, KEGG}, {SIGNOR}]",4,"[1, -1]",2
44,CDK5,MAPT,"[1, -1, 1, -1]","[{CA1, Wang, KEGG-MEDICUS, Cui2007, KEGG}, {SP...",4,"[1, -1]",2
62,GRB2,PAK1,"[1, 0]","[{SPIKE_LC, SPIKE}, {Adhesome, Wang}]",2,"[0, 1]",2
75,MAPK1,MAPK8,"[-1, 0]","[{Cui2007, Wang}, {Adhesome}]",2,"[0, -1]",2
76,MAPK1,MAPT,"[1, -1, 1, -1]","[{Cui2007, CA1, Wang}, {SIGNOR}, {Cui2007, CA1...",4,"[1, -1]",2
77,MAPK1,PAK1,"[1, -1]","[{Adhesome, Wang}, {SIGNOR}]",2,"[1, -1]",2


In [ ]:
# if we have only a 1 and a 0, we keep the 1, if we have only a -1 and a 0 in effect_unique, we keep the -1
df_graph_all_nodes_only_merge_17genes_3_unique_2["effect_unique"] = df_graph_all_nodes_only_merge_17genes_3_unique_2["effect_unique"].apply(lambda x: [1] if (0 in x and 1 in x and len(x)==2) else [-1] if (0 in x and -1 in x and len(x)==2) else x)
df_graph_all_nodes_only_merge_17genes_3_unique_2

/var/folders/zg/y03kdtc553xg5q1ct59rj44c0000gn/T/ipykernel_96270/2504061944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graph_all_nodes_only_merge_17genes_3_unique_2["effect_unique"] = df_graph_all_nodes_only_merge_17genes_3_unique_2["effect_unique"].apply(lambda x: [1] if (0 in x and 1 in x and len(x)==2) else [-1] if (0 in x and -1 in x and len(x)==2) else x)


,HGNC_a,HGNC_b,effect,sources,effect_size,effect_unique,effect_size_unique
5,ATM,TP53,"[1, -1, 0]","[{SPIKE, SIGNOR, SPIKE_LC, Cui2007, KEGG}, {Cu...",3,"[0, 1, -1]",3
12,AURKA,TP53,"[1, -1]","[{Wang, SIGNOR}, {SPIKE, SPIKE_LC, SIGNOR}]",2,"[1, -1]",2
18,AURKB,H3C7,"[1, -1, 1, -1]","[{SIGNOR}, {SPIKE_LC, SPIKE}, {SIGNOR}, {SPIKE...",4,"[1, -1]",2
22,AURKB,KIF2C,"[1, -1, 1, -1]","[{SPIKE, SPIKE_LC, SIGNOR}, {Wang, SIGNOR}, {S...",4,"[1, -1]",2
27,CAMK2A,MAPT,"[1, -1, 1, -1]","[{Wang, KEGG}, {SIGNOR}, {Wang, KEGG}, {SIGNOR}]",4,"[1, -1]",2
44,CDK5,MAPT,"[1, -1, 1, -1]","[{CA1, Wang, KEGG-MEDICUS, Cui2007, KEGG}, {SP...",4,"[1, -1]",2
62,GRB2,PAK1,"[1, 0]","[{SPIKE_LC, SPIKE}, {Adhesome, Wang}]",2,[1],2
75,MAPK1,MAPK8,"[-1, 0]","[{Cui2007, Wang}, {Adhesome}]",2,[-1],2
76,MAPK1,MAPT,"[1, -1, 1, -1]","[{Cui2007, CA1, Wang}, {SIGNOR}, {Cui2007, CA1...",4,"[1, -1]",2
77,MAPK1,PAK1,"[1, -1]","[{Adhesome, Wang}, {SIGNOR}]",2,"[1, -1]",2


#### add information on the nodes: in the 17 DE genes, in the intersection of 2 (blue, green, purple?), in the MAPs (yellow) + up/down regulated (change borders)

In [ ]:
# faire une datafame avec 1 première colonne id_a et une deuxième colonne id_a et une troisième colonne effect = 2
df_nodes = pd.DataFrame({"id_a": all_nodes, "id_b": all_nodes,"HGNC_a": all_nodes, "HGNC_b": all_nodes})
df_nodes = df_nodes.drop_duplicates()
df_nodes["effect"] = 2
df_nodes

,id_a,id_b,HGNC_a,HGNC_b,effect
0,AURKA,AURKA,AURKA,AURKA,2
1,TTBK1,TTBK1,TTBK1,TTBK1,2
2,KIF4A,KIF4A,KIF4A,KIF4A,2
3,ATM,ATM,ATM,ATM,2
4,GTSE1,GTSE1,GTSE1,GTSE1,2
5,CIT,CIT,CIT,CIT,2
6,CAMK2B,CAMK2B,CAMK2B,CAMK2B,2
7,FYN,FYN,FYN,FYN,2
8,KIF15,KIF15,KIF15,KIF15,2
9,ASPM,ASPM,ASPM,ASPM,2


#### Adding color codes
* 0: nothing (no color) light mauve
* 1: MT-related genes (411) very light blue (to be updated with CIT and PRC1)
* 2: MT-related genes (280) light blue
* 3: common to 2 studies: R02-R04 (17+9) or R02-MDA (17+2) or R04 MDA (17+14) medium blue
* 4: Common R02 R04 MDA (17) dark blue

In [ ]:
df_411 = pd.read_csv('../data/411-MT-related-genes.csv',sep=';')
df_411.head()

,Symbol,Entrez Gene Name,Entrez Gene ID for Human,Unnamed: 3,Unnamed: 4
0,AGTPBP1,ATP/GTP binding protein 1,23287,NaN,NaN
1,AJUBA,AJUBA LIM protein,84962,NaN,NaN
2,AMER2,APC membrane recruitment protein 2,219287,NaN,NaN
3,APC,adenomatous polyposis coli,324,NaN,NaN
4,APC2,adenomatosis polyposis coli 2,10297,NaN,NaN


In [ ]:
# we add a column in df_nodes with 2 if the node is in df_280, 1 if the node is in df_411 and 0 otherwise
df_nodes['colors'] = [2 if gene in list(df_280['Symbol']) else 1 if gene in (list(df_411['Symbol'])+["CIT","PRC1"]) else 0 for gene in df_nodes['HGNC_a']]
df_nodes.head()

,id_a,id_b,HGNC_a,HGNC_b,effect,colors
0,AURKA,AURKA,AURKA,AURKA,2,2
1,TTBK1,TTBK1,TTBK1,TTBK1,2,2
2,KIF4A,KIF4A,KIF4A,KIF4A,2,2
3,ATM,ATM,ATM,ATM,2,0
4,GTSE1,GTSE1,GTSE1,GTSE1,2,2


In [ ]:
df_genes_common = pd.read_csv('../data/Listes-gènes-communs.csv',sep=';')
df_genes_common.head()

,Communs R02 R04 MDA (17),Communs R02-R04 (17+9),Communs R02-MDA (17+2),Communs R04-MDA (17+14),Olivier
0,ASPM,AURKA,EML4,AGTPBP1,ASPM
1,AURKB,BIRC5,SEL1L,AR,GTSE1
2,GTSE1,FNTB,NaN,BAG3,KIF20A
3,KIF11,KIF23,NaN,CCND1,KIF2C
4,KIF14,KIFC1,NaN,DCTN2,MAPT


In [ ]:
# we complete the 'colors' column in df_nodes with 2 if the node is in df_genes_common['Communs R02 R04 MDA (17)'], 3 if the node is in df_genes_common['Communs R02-R04 (17+9)'] and 4 if the node is in df_genes_common['Communs R02-R04 (17+9)']
df_nodes['colors'] = [4 if df_nodes.loc[i,'HGNC_a'] in list(df_genes_common['Communs R02 R04 MDA (17)'])else 3 if  df_nodes.loc[i,'HGNC_a'] in list(df_genes_common['Communs R02-R04 (17+9)'])+list(df_genes_common['Communs R02-MDA (17+2)'])+list(df_genes_common['Communs R04-MDA (17+14)']) else  df_nodes.loc[i,'colors'] for i in df_nodes.index]

df_nodes.head()

,id_a,id_b,HGNC_a,HGNC_b,effect,colors
0,AURKA,AURKA,AURKA,AURKA,2,3
1,TTBK1,TTBK1,TTBK1,TTBK1,2,2
2,KIF4A,KIF4A,KIF4A,KIF4A,2,4
3,ATM,ATM,ATM,ATM,2,0
4,GTSE1,GTSE1,GTSE1,GTSE1,2,4


In [ ]:
df_results

,HGNC_a,p-value,log2FC,log2FC2
0,ASPM,9.670000e-20,12.06,14.66
1,ATM,2.680000e-07,0.72,0.77
2,AURKA,5.290000e-20,8.21,7.01
3,AURKB,5.980000e-19,8.05,8.05
4,CAMK2A,4.430000e-08,0.24,2.25
...,...,...,...,...
119,STAU1,2.480000e-09,1.53,1.37
120,SYK,2.470000e-02,1.24,1.17
121,TTBK1,4.450000e-08,3.05,2.00
122,VHL,5.110000e-03,1.24,1.13


In [ ]:
df_nodes = df_nodes.copy()
df_nodes = df_nodes.merge(df_results, on="HGNC_a", how="left")  
present = df_nodes["p-value"].notna()  

p = (df_nodes["p-value"] < 0.05)
fc = df_nodes["log2FC2"]

conds = [
    p & (fc >= 5),         
    p & (fc >= 2),       
    p & (fc <= 0.5),        
    p & (fc >= 1.5),        
    p & (fc <= (2/3)),      
    present                
]
choices = [3, 2, -2, 1, -1, 0]  

df_nodes["up_down_normal_tumor"] = np.select(conds, choices, default=5).astype(int)

df_nodes = df_nodes.drop(columns=["p-value","log2FC","log2FC2"])
df_nodes.head()


,id_a,id_b,HGNC_a,HGNC_b,effect,colors,up_down_normal_tumor
0,AURKA,AURKA,AURKA,AURKA,2,3,3
1,TTBK1,TTBK1,TTBK1,TTBK1,2,2,2
2,KIF4A,KIF4A,KIF4A,KIF4A,2,4,3
3,ATM,ATM,ATM,ATM,2,0,0
4,GTSE1,GTSE1,GTSE1,GTSE1,2,4,3
